In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [7]:
STORM_NUMBER = 2
TITLE = "15-16 November 2022"
SUBTITLE = "8 AM 16 November 2022"
SETPOINT_LOCS = {}
sts = utc(2022, 11, 15, 11)
ets = utc(2022, 11, 16, 15)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [4]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2022-2023 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [5]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [17]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = ['CIYI4', 154, 'ESTI4', 158, 'IA-SR-45', 'SWEI4', 'LWDI4', 839, 'SANI4', 'IA-WT-1']
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [
        ('IA-BH-6', 2.75),
        ('IA-SH-6', 1),
        ('IA-ID-4', 1.5),
    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=5)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='221116.png')
    mp.close()

main()

     state  wfo   val        lon       lat                               geo  \
0       MN  DLH  21.0  -91.18000  47.42000     POINT (662053.673 305556.954)   
1       MN  DLH  19.7  -90.12000  47.84000     POINT (735259.718 361255.996)   
2       MN  DLH  18.5  -89.96000  47.85000     POINT (746945.963 363845.026)   
3       MN  DLH  18.5  -91.20000  47.38000     POINT (661057.574 300966.020)   
4       SD  UNR  17.0 -103.77000  44.35000    POINT (-299636.667 -65322.968)   
...    ...  ...   ...        ...       ...                               ...   
3721    WY  RIW   0.0 -110.77060  41.83690   POINT (-889454.069 -293205.855)   
3723    WY  RIW   0.0 -110.27249  41.31544   POINT (-855604.157 -356128.190)   
3725    WY  RIW   0.0 -110.83280  44.45690     POINT (-856606.668 -3094.354)   
3726    WY  RIW   0.0 -106.98940  43.03530   POINT (-567291.365 -194166.736)   
3727    AK  AFC   0.0 -149.10810  60.95920  POINT (-2435509.358 2611654.110)   

      used_for_analysis  nwsli  plotme 